In [2]:
from watttime.shared_anniez.alg.optCharger import OptCharger, Moer
from watttime import WattTimeHistorical, WattTimeForecast

In [3]:
import pandas as pd
from datetime import datetime
import os

In [4]:
region = "CAISO_NORTH"
username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

In [5]:
hist_data = WattTimeForecast(username, password)
actual_data = WattTimeHistorical(username, password)

In [6]:
generated_data = pd.read_csv('dummy_eval_results_1K_simulated_users.csv').iloc[:10]

# get user data and convert types
generated_data['charge_rate_per_window'] = generated_data['user_type'].apply(lambda x: float(x.split('_')[0][1:])/12).astype('float64')
generated_data['charge_needed'] = generated_data['user_type'].apply(lambda x: x.split('_')[1][2:]).astype('float64')
generated_data['avg_charge_window'] = generated_data['user_type'].apply(
    lambda x: x.split('_')[2][5:]
).astype('float64')
generated_data['avg_charge_window'] = generated_data['avg_charge_window'].apply(lambda x: pd.Timedelta(seconds=x))
generated_data['length_plugged_in'] = generated_data['length_plugged_in'].apply(lambda x: pd.Timedelta(seconds=x))

# Get per-row historical fcsts at 'plug in time'
def get_historical_fcst_data(plug_in_time, avg_charge_window):
    plug_in_time = pd.Timestamp(plug_in_time)
    horizon = round(avg_charge_window.total_seconds() / 3600) + 1
    return hist_data.get_historical_forecast_pandas(
        start=plug_in_time - pd.Timedelta(minutes=5), 
        end=plug_in_time, 
        horizon_hours=horizon,
        region=region
    )

generated_data['moer_data'] = generated_data.apply(lambda x: get_historical_fcst_data(x.plug_in_time, x.avg_charge_window), axis=1)
generated_data

,distinct_dates,user_type,plug_in_time,unplug_time,initial_charge,total_seconds_to_95,full_charge_time,length_plugged_in,session_charge,final_perc_charged,final_charge_time,uncharged,sum_moer,count_moer_intervals,avg_moer,Unnamed: 15,charge_rate_per_window,charge_needed,avg_charge_window,moer_data
0,2023-04-01 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-01 17:26:48+00:00,2023-04-01 19:28:24.229988+00:00,0.094096,5042.055008,2023-04-01 18:50:50.055007548+00:00,0 days 02:01:36.229988,30.81255838,0.95,2023-04-01 18:50:50.055007548+00:00,False,23750.07641,17,1397.063318,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
1,2023-04-02 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-02 18:25:11+00:00,2023-04-03 00:31:01.351065+00:00,0.005743,5562.532798,2023-04-02 19:57:53.532797545+00:00,0 days 06:05:50.351070,33.99325598,0.95,2023-04-02 19:57:53.532797545+00:00,False,25166.87962,18,1398.159979,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
2,2023-04-03 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-03 18:35:30+00:00,2023-04-04 00:19:21.063471+00:00,0.198815,4425.161733,2023-04-03 19:49:15.161733450+00:00,0 days 05:43:51.063470,27.04265504,0.95,2023-04-03 19:49:15.161733450+00:00,False,19581.76028,14,1398.697163,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
3,2023-04-04 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-04 19:58:18+00:00,2023-04-05 05:16:18.185832+00:00,0.003162,5577.738501,2023-04-04 21:31:15.738501330+00:00,0 days 09:18:00.185830,34.08617973,0.95,2023-04-04 21:31:15.738501330+00:00,False,26584.04536,19,1399.160282,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
4,2023-04-05 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-05 19:00:34+00:00,2023-04-05 23:59:25.184713+00:00,0.186789,4496.008637,2023-04-05 20:15:30.008637113+00:00,0 days 04:58:51.184710,27.47560834,0.95,2023-04-05 20:15:30.008637113+00:00,False,21009.55466,15,1400.636978,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
5,2023-04-06 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-06 20:40:36+00:00,2023-04-07 04:45:51.268066+00:00,0.250131,4122.864295,2023-04-06 21:49:18.864294813+00:00,0 days 08:05:15.268070,25.1952818,0.95,2023-04-06 21:49:18.864294813+00:00,False,18183.99397,13,1398.768767,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
6,2023-04-07 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-07 20:13:02+00:00,2023-04-08 04:01:14.473801+00:00,0.035883,5384.979547,2023-04-07 21:42:46.979547319+00:00,0 days 07:48:12.473800,32.90820834,0.95,2023-04-07 21:42:46.979547319+00:00,False,25162.08668,18,1397.893704,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
7,2023-04-08 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-08 17:51:19+00:00,2023-04-09 03:25:18.647995+00:00,0.184693,4508.352057,2023-04-08 19:06:27.352057017+00:00,0 days 09:33:59.648000,27.55104035,0.95,2023-04-08 19:06:27.352057017+00:00,False,20985.51454,15,1399.034303,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
8,2023-04-09 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-09 20:30:27+00:00,2023-04-10 03:30:14.863872+00:00,0.243165,4163.903003,2023-04-09 21:39:50.903003422+00:00,0 days 06:59:47.863870,25.44607391,0.95,2023-04-09 21:39:50.903003422+00:00,False,18192.36008,13,1399.412314,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...
9,2023-04-10 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-10 20:31:33+00:00,2023-04-11 05:03:53.897268+00:00,0.026754,5438.755463,2023-04-10 22:02:11.755463258+00:00,0 days 08:32:20.897270,33.23683894,0.95,2023-04-10 22:02:11.755463258+00:00,False,25212.42676,18,1400.690376,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...


In [7]:
data_with_moer = generated_data.copy()

# Set up OptCharger based on moer fcsts and get info on projected schedule
def get_schedule_and_cost(charge_rate_per_window, charge_needed, avg_charge_window, moer_data):
    charger = OptCharger(charge_rate_per_window) # charge rate needs to be an int
    moer = Moer(moer_data['value'])

    charger.fit(
        totalCharge=charge_needed, # also currently an int value
        totalTime = int(avg_charge_window.total_seconds() / 300),
        moer=moer
    )
    return charger

data_with_moer['charger']= data_with_moer.apply(lambda x: get_schedule_and_cost(x.charge_rate_per_window, x.charge_needed, x.avg_charge_window, x.moer_data), axis = 1)
data_with_moer['charging_schedule'] = data_with_moer['charger'].apply(lambda x: x.get_schedule())
data_with_moer['projected_charging_cost_to_full'] = data_with_moer['charger'].apply(lambda  x: x.get_total_cost())
data_with_moer

Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!
Simplified fit!


,distinct_dates,user_type,plug_in_time,unplug_time,initial_charge,total_seconds_to_95,full_charge_time,length_plugged_in,session_charge,final_perc_charged,...,count_moer_intervals,avg_moer,Unnamed: 15,charge_rate_per_window,charge_needed,avg_charge_window,moer_data,charger,charging_schedule,projected_charging_cost_to_full
0,2023-04-01 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-01 17:26:48+00:00,2023-04-01 19:28:24.229988+00:00,0.094096,5042.055008,2023-04-01 18:50:50.055007548+00:00,0 days 02:01:36.229988,30.81255838,0.95,...,17,1397.063318,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",16974.100000
1,2023-04-02 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-02 18:25:11+00:00,2023-04-03 00:31:01.351065+00:00,0.005743,5562.532798,2023-04-02 19:57:53.532797545+00:00,0 days 06:05:50.351070,33.99325598,0.95,...,18,1398.159979,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",15926.716667
2,2023-04-03 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-03 18:35:30+00:00,2023-04-04 00:19:21.063471+00:00,0.198815,4425.161733,2023-04-03 19:49:15.161733450+00:00,0 days 05:43:51.063470,27.04265504,0.95,...,14,1398.697163,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",19325.533333
3,2023-04-04 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-04 19:58:18+00:00,2023-04-05 05:16:18.185832+00:00,0.003162,5577.738501,2023-04-04 21:31:15.738501330+00:00,0 days 09:18:00.185830,34.08617973,0.95,...,19,1399.160282,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",63703.200000
4,2023-04-05 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-05 19:00:34+00:00,2023-04-05 23:59:25.184713+00:00,0.186789,4496.008637,2023-04-05 20:15:30.008637113+00:00,0 days 04:58:51.184710,27.47560834,0.95,...,15,1400.636978,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",31333.683333
5,2023-04-06 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-06 20:40:36+00:00,2023-04-07 04:45:51.268066+00:00,0.250131,4122.864295,2023-04-06 21:49:18.864294813+00:00,0 days 08:05:15.268070,25.1952818,0.95,...,13,1398.768767,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",48613.400000
6,2023-04-07 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-07 20:13:02+00:00,2023-04-08 04:01:14.473801+00:00,0.035883,5384.979547,2023-04-07 21:42:46.979547319+00:00,0 days 07:48:12.473800,32.90820834,0.95,...,18,1397.893704,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",49283.483333
7,2023-04-08 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-08 17:51:19+00:00,2023-04-09 03:25:18.647995+00:00,0.184693,4508.352057,2023-04-08 19:06:27.352057017+00:00,0 days 09:33:59.648000,27.55104035,0.95,...,15,1399.034303,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",15976.400000
8,2023-04-09 00:00:00+00:00,r22_tc118_avglc23883_sdlc7140,2023-04-09 20:30:27+00:00,2023-04-10 03:30:14.863872+00:00,0.243165,4163.903003,2023-04-09 21:39:50.903003422+00:00,0 days 06:59:47.863870,25.44607391,0.95,...,13,1399.412314,NaN,1.833333,118.0,0 days 06:38:03,point_time value ...,<watttime.shared_anniez.alg.optCharger.OptChar...,"[1.8333333333333333, 1.8333333333333333, 1.833...",45278.200000
9,2023-04-10 00:00:00+00:00,r22_tc118_avg

In [8]:
data_with_charging_schedule = data_with_moer.copy()

def get_amt_charged_on_unplug(length_plugged_in, charging_schedule, charge_needed):
    intervals_charged = int(length_plugged_in.total_seconds() / 300)
    total_amt_charged = sum(charging_schedule[:intervals_charged])
    return total_amt_charged / charge_needed

def get_actual_moer(plug_in_time, length_plugged_in):
    return actual_data.get_historical_pandas(
        start=plug_in_time, 
        end=pd.Timestamp(plug_in_time) + length_plugged_in, 
        region=region
    )

def get_actual_moer_on_unplug(length_plugged_in, actual_moer, charging_schedule):
    intervals_charged = min(int(length_plugged_in.total_seconds() / 300), len(charging_schedule))
    return sum(charging_schedule[:intervals_charged] * actual_moer["value"][:intervals_charged])

# Assuming we use the schedule created at plug-in time, calculate the percentage charge on unplug and the cost of the charge used    
data_with_charging_schedule['actual_moer'] = data_with_charging_schedule.apply(lambda x: get_actual_moer(x.plug_in_time, x.length_plugged_in), axis=1)
data_with_charging_schedule['optimized_charge_pct_on_unplug'] = data_with_charging_schedule.apply(lambda x: get_amt_charged_on_unplug(x.length_plugged_in, x.charging_schedule, x.charge_needed), axis=1)
data_with_charging_schedule['optimized_charge_cost_on_unplug'] = data_with_charging_schedule.apply(lambda x: get_actual_moer_on_unplug(x.length_plugged_in, x.actual_moer, x.charging_schedule), axis=1)

In [9]:
# NOTE: sum_moer right now is not based on actual MOER data. TODO: Add comparison between the "ASAP" methodology (percent charge and cost)
data_with_charging_schedule[["final_perc_charged", "sum_moer", "optimized_charge_pct_on_unplug", "optimized_charge_cost_on_unplug", "projected_charging_cost_to_full"]]

,final_perc_charged,sum_moer,optimized_charge_pct_on_unplug,optimized_charge_cost_on_unplug,projected_charging_cost_to_full
0,0.95,23750.07641,0.372881,4319.333333,16974.100000
1,0.95,25166.87962,1.009887,12041.333333,15926.716667
2,0.95,19581.76028,1.009887,12974.500000,19325.533333
3,0.95,26584.04536,1.009887,22093.500000,63703.200000
4,0.95,21009.55466,0.916667,10538.000000,31333.683333
5,0.95,18183.99397,1.009887,39224.166667,48613.400000
6,0.95,25162.08668,1.009887,36598.833333,49283.483333
7,0.95,20985.51454,1.009887,12015.666667,15976.400000
8,0.95,18192.36008,1.009887,40547.833333,45278.200000
9,0.95,25212.42676,1.009887,54781.833333,62458.550000
